In [1]:
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib qt 

In [2]:
"""
***Aestetics***
"""
# load color palette
my_palette = sns.color_palette().as_hex()
sns.color_palette().as_hex()

['#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f',
 '#bcbd22',
 '#17becf']

In [3]:
""" 
Load files
"""
el_1 = "POz"
el_2 = "PO7"
el_3 = "O1"
el_4 = "PO4"
el_5 = "PO8"
el_6 = "O2"
# el_2_path = "C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/variable-electrode/"
el_1_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_1}/"
el_2_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_2}/"
el_3_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_3}/"
el_4_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_4}/"
el_5_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_5}/"
el_6_path = f"C:/Users/mvmigem/Documents/data/project_1/overlap_corrected/{el_6}/"

el_1_dir_list = glob.glob(el_1_path+'/*.csv')
el_2_dir_list = glob.glob(el_2_path+'/*.csv')
el_3_dir_list = glob.glob(el_3_path+'/*.csv')
el_4_dir_list = glob.glob(el_4_path+'/*.csv')
el_5_dir_list = glob.glob(el_5_path+'/*.csv')
el_6_dir_list = glob.glob(el_6_path+'/*.csv')
excuded_pp = [3,14,20]
subject_list = []

for i in el_1_dir_list:
    sub = int(i.split(f'corrected_{el_1}_evoked_')[1].split('.csv')[0])
    if sub not in excuded_pp:
        subject_list.append(sub)

# Create big df for each var and fix set of rERP's
el_1_loaded_dfs = []
el_2_loaded_dfs = []
el_3_loaded_dfs = []
el_4_loaded_dfs = []
el_5_loaded_dfs = []
el_6_loaded_dfs = []

for el1_p,el2_p,el3_p,el4_p,el5_p,el6_p in zip(el_1_dir_list,el_2_dir_list,el_3_dir_list,el_4_dir_list,el_5_dir_list,el_6_dir_list):
    el_1_rerp_df = pd.read_csv(el1_p)
    el_1_loaded_dfs.append(el_1_rerp_df)
    el_2_rerp_df = pd.read_csv(el2_p)
    el_2_loaded_dfs.append(el_2_rerp_df)
    el_3_rerp_df = pd.read_csv(el3_p)
    el_3_loaded_dfs.append(el_3_rerp_df)
    el_4_rerp_df = pd.read_csv(el4_p)
    el_4_loaded_dfs.append(el_4_rerp_df)
    el_5_rerp_df = pd.read_csv(el5_p)
    el_5_loaded_dfs.append(el_5_rerp_df)
    el_6_rerp_df = pd.read_csv(el6_p)
    el_6_loaded_dfs.append(el_6_rerp_df)

df_1 = pd.concat(el_1_loaded_dfs,ignore_index=True)
df_2 = pd.concat(el_2_loaded_dfs,ignore_index=True)
df_3 = pd.concat(el_3_loaded_dfs,ignore_index=True)
df_4 = pd.concat(el_4_loaded_dfs,ignore_index=True)
df_5 = pd.concat(el_5_loaded_dfs,ignore_index=True)
df_6 = pd.concat(el_6_loaded_dfs,ignore_index=True)

df_1['visual_field'] = np.where(df_1['position'].isin([1,2]),'up','down')
df_2['visual_field'] = np.where(df_2['position'].isin([1,2]),'up','down')
df_3['visual_field'] = np.where(df_3['position'].isin([1,2]),'up','down')
df_4['visual_field'] = np.where(df_4['position'].isin([1,2]),'up','down')
df_5['visual_field'] = np.where(df_5['position'].isin([1,2]),'up','down')
df_6['visual_field'] = np.where(df_6['position'].isin([1,2]),'up','down')

df_1 = df_1[df_1['time'] < .6]
df_2 = df_2[df_2['time'] < .6]
df_3 = df_3[df_3['time'] < .6]
df_4 = df_4[df_4['time'] < .6]
df_5 = df_5[df_5['time'] < .6]
df_6 = df_6[df_6['time'] < .6]

df_1.reset_index(inplace=True)
df_2.reset_index(inplace=True)
df_3.reset_index(inplace=True)
df_4.reset_index(inplace=True)
df_5.reset_index(inplace=True)
df_6.reset_index(inplace=True)


In [4]:
""" 
baseline correction
"""
for df in [df_1,df_2,df_3,df_4,df_5,df_6]:
    baseline_period = df['time'] <0
    baseline_means = df[baseline_period].groupby(['subject','attention','expectation','visual_field'])['yhat'].mean()

    df['baseline_corrected'] = df.apply(
        lambda row: row['yhat'] - baseline_means[(row['subject'], row['attention'], row['expectation'],row['visual_field'])],
        axis=1
    )
    # Correct dataframe for subjects 8 and 9 having truncated datasets (lost trials)
    df.loc[(df['subject'] == 9) & (df['attention'] == 'attended') & (df['expectation'] == 'odd'), 
        ['yhat', 'baseline_corrected']] = np.nan
    df.loc[(df['subject'] == 9) & (df['attention'] == 'attended') & (df['expectation'] == 'regular'), 
        ['yhat', 'baseline_corrected']] = np.nan
    df.loc[(df_1['subject'] == 8) & (df['attention'] == 'attended') & (df['expectation'] == 'regular'), 
        ['yhat', 'baseline_corrected']] = np.nan
    df.loc[(df['subject'] == 8) & (df['attention'] == 'attended') & (df['expectation'] == 'odd'), 
        ['yhat', 'baseline_corrected']] = np.nan
    

In [7]:
""" 
Plotting rERP's
"""
fig, axes = plt.subplots(3,2,figsize=(16, 12),sharey=True,sharex=True)

sns.lineplot(data=df_1[df_1['visual_field']=='up'],
            x='time',y='baseline_corrected',
            hue='attention',style='expectation',
            errorbar='se',ax=axes[0,0])
sns.lineplot(data=df_1[df_1['visual_field']=='down'],
            x='time',y='baseline_corrected',
            hue='attention',style='expectation',
            errorbar='se', ax = axes[1,0])
sns.lineplot(data=df_3,
            x='time',y='baseline_corrected',
            hue='attention',style='expectation',
            errorbar='se',ax=axes[0,1])
sns.lineplot(data=df_4,
            x='time',y='baseline_corrected',
            hue='attention',style='expectation',
            errorbar='se', ax = axes[1,1])
sns.lineplot(data=df_5,
            x='time',y='baseline_corrected',
            hue='attention',style='expectation',
            errorbar='se', ax = axes[2,0])
sns.lineplot(data=df_6,
            x='time',y='baseline_corrected',
            hue='attention',style='expectation',
            errorbar='se', ax = axes[2,1])
axes[0,0].set_title(f'{el_1}')
axes[1,0].set_title(f'{el_2}')
axes[0,1].set_title(f'{el_3}')
axes[1,1].set_title(f'{el_4}')
axes[2,0].set_title(f'{el_5}')
axes[2,1].set_title(f'{el_6}')

for i, ax in enumerate(axes.flat):
    ax.axhline(y=0, lw=1, c='black' )


plt.tight_layout()
plt.show()


In [6]:
""" 
Select the C1 to analyse
"""
##  C1 time window
# Window search
c1_tmin = 0.06
c1_tmax = 0.09
c1_timwid = 0.015
c1_sampwid = int(c1_timwid*512)
for df in [df_1,df_2]:
    # Find indices of df with peak in window DF 1
    slect = df[(df['time']>c1_tmin) & (df['time']<c1_tmax)]
    slect['abs_corr'] = slect['baseline_corrected'].abs()
    maxidx_c1 = np.array(slect.groupby(['subject','attention','expectation','visual_field'])['abs_corr'].idxmax())
    maxidx_c1 = maxidx_c1[~np.isnan(maxidx_c1)]
    maxidx_c1 = maxidx_c1.astype(int)
    # Select window around it
    winidx_c1 =[]
    for i, idx in enumerate(maxidx_c1):
        win_li = np.arange(idx - c1_sampwid, idx + c1_sampwid +1)
        winidx_c1 = np.concatenate([winidx_c1,win_li],axis=0)

# Apply to df
c1_win_df_1 = df_1.iloc[winidx_c1]
c1_means_df_1 = c1_win_df_1.groupby(['subject','attention','expectation','visual_field'])['baseline_corrected'].mean().reset_index()


# Apply to df
c1_win_df_2 = df_2.iloc[winidx_c1]
c1_means_df_2 = c1_win_df_2.groupby(['subject','attention','expectation','visual_field'])['baseline_corrected'].mean().reset_index()

C:\Users\mvmigem\AppData\Local\Temp\ipykernel_24828\1716124993.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slect['abs_corr'] = slect['baseline_corrected'].abs()
C:\Users\mvmigem\AppData\Local\Temp\ipykernel_24828\1716124993.py:14: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  maxidx_c1 = np.array(slect.groupby(['subject','attention','expectation','visual_field'])['abs_corr'].idxmax())
C:\Users\mvmigem\AppData\Local\Temp\ipykernel_24828\1716124993.py:14: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  maxidx_c1 = np.array(

In [10]:
fig, axes = plt.subplots(2,1,figsize=(14, 10),sharey=True,sharex=True)

sns.barplot(c1_means_df_1,y='baseline_corrected',x='attention',hue='expectation',edgecolor= 'none', palette= [my_palette[4],my_palette[8]],
            errorbar='se', errwidth= 5,
            capsize=.1,width=.4, linewidth=0.7,ax=axes[0])
sns.barplot(c1_means_df_2,y='baseline_corrected',x='attention',hue='expectation',edgecolor= 'none', palette= [my_palette[4],my_palette[8]],
            errorbar='se', errwidth= 5,
            capsize=.1,width=.4, linewidth=0.7,ax=axes[1])
plt.tight_layout()
plt.show()

C:\Users\mvmigem\AppData\Local\Temp\ipykernel_24828\2439826541.py:3: FutureWarning: 

The `errwidth` parameter is deprecated. And will be removed in v0.15.0. Pass `err_kws={'linewidth': 5}` instead.

  sns.barplot(c1_means_df_1,y='baseline_corrected',x='attention',hue='expectation',edgecolor= 'none', palette= [my_palette[4],my_palette[8]],
C:\Users\mvmigem\AppData\Local\Temp\ipykernel_24828\2439826541.py:6: FutureWarning: 

The `errwidth` parameter is deprecated. And will be removed in v0.15.0. Pass `err_kws={'linewidth': 5}` instead.

  sns.barplot(c1_means_df_2,y='baseline_corrected',x='attention',hue='expectation',edgecolor= 'none', palette= [my_palette[4],my_palette[8]],


In [12]:
fig, axes = plt.subplots(2,1,figsize=(8, 8))
sns.lineplot(data=df_1[df_1['visual_field']=='up'],
            x='time',y='baseline_corrected',
            hue='attention',style='expectation',
            errorbar='se',ax=axes[0])
sns.lineplot(data=df_1[df_1['visual_field']=='down'],
            x='time',y='baseline_corrected',
            hue='attention',style='expectation',
            errorbar='se', ax = axes[1])

axes[0].axhline(y=0, lw=1, c='black' )
axes[0].axhline(y=0, lw=1, c='black' )
axes[0].axvline(x=.05,ls='--',lw=1, c='black' )
axes[0].axvline(x=.1,ls='--',lw=1, c='black' )
axes[1].axvline(x=.05,ls='--',lw=1, c='black' )
axes[1].axvline(x=.1,ls='--',lw=1, c='black' )

plt.tight_layout()
plt.show()

In [ ]:
destination_folder = r"C:\Users\mvmigem\Documents\data\project_1\compiled_dataframes/"
c1_means_df_2.to_csv(destination_folder + "c1_oc_var_df.csv")

In [ ]:
""" 
Select P3 to analyse
"""
p3_df = pd.DataFrame({'attention':df_1.attention,'expectation':df_1.expectation,'position':df_1.position,
                      'time':df_1.time, 'visual_field':df_1.visual_field,'subject':df_1.subject,
                      el_1:df_1.baseline_corrected, el_2:df_2.baseline_corrected,
                      el_3:df_3.baseline_corrected, el_4:df_4.baseline_corrected,
                      el_5:df_5.baseline_corrected, el_6:df_6.baseline_corrected})

p3_df['P3a'] = p3_df[['Fz','FCz']].mean(axis=1)
p3_df['P3b'] = p3_df[['Cz','CPz']].mean(axis=1)
# Window selection
p3a_tmin = 0.3
p3a_tmax = 0.45
p3b_tmin = 0.35
p3b_tmax = 0.5
p3_timwid = 0.05
p3_sampwid = int(p3_timwid*512)

# Select the place to search for peak
p3a_slect = p3_df[(p3_df['time']>p3a_tmin) & (df['time']<p3a_tmax)]
collaps_p3a = p3a_slect.groupby(['subject','time'])['P3a'].max().reset_index()
p3a_peak_lat = collaps_p3a.loc[collaps_p3a.groupby('subject')['P3a'].idxmax(), ['subject', 'time']]

p3b_slect = p3_df[(p3_df['time']>p3b_tmin) & (df['time']<p3b_tmax)]
collaps_p3b = p3b_slect.groupby(['subject','time'])['P3b'].max().reset_index()
p3b_peak_lat = collaps_p3b.loc[collaps_p3b.groupby('subject')['P3b'].idxmax(), ['subject', 'time']]

winidx_p3a = []
winidx_p3b = []

for df,winidx in zip([p3a_peak_lat,p3b_peak_lat],[winidx_p3a,winidx_p3b]):
    # Find time point in main df
    idx_p3 =[]
    for id,row in df.iterrows():
        sub = row.subject
        time_point = row.time
        # Find matching rows in df
        matches = p3_df[(p3_df['subject'] == sub) & (p3_df['time'] == time_point)].index.tolist()
        idx_p3.extend(matches)

    # Select window around it
    for i, idx in enumerate(idx_p3):
        win_li = np.arange(idx - p3_sampwid, idx + p3_sampwid +1)
        winidx.extend(win_li)

# Apply to df
p3a_win = p3_df.iloc[winidx_p3a]
p3a_means= p3a_win.groupby(['subject','attention','expectation'])['P3a'].mean().reset_index()

# Apply to df
p3b_win = p3_df.iloc[winidx_p3b]
p3b_means = p3b_win.groupby(['subject','attention','expectation','visual_field'])['P3b'].mean().reset_index()


In [ ]:
fig, axes = plt.subplots(2,1,figsize=(14, 10),sharey=True,sharex=True)

sns.barplot(p3a_means,y='P3a',x='attention',hue='expectation',edgecolor= 'none', palette= [my_palette[4],my_palette[8]],
            errorbar='se', errwidth= 5,
            capsize=.1,width=.4, linewidth=0.7,ax=axes[0])
sns.barplot(p3b_means,y='P3b',x='attention',hue='expectation',edgecolor= 'none', palette= [my_palette[4],my_palette[8]],
            errorbar='se', errwidth= 5,
            capsize=.1,width=.4, linewidth=0.7,ax=axes[1])
plt.tight_layout()
plt.show()

In [ ]:
destination_folder = r"C:\Users\mvmigem\Documents\data\project_1\compiled_dataframes/"
p3a_means.to_csv(destination_folder + "p3a_oc_df.csv")
p3b_means.to_csv(destination_folder + "p3b_oc_df.csv")

In [ ]:
for i in subject_list:
    fig, axes = plt.subplots(2,1,figsize=(8, 8))
    sns.lineplot(data=p3a_win[p3_df['subject']==i],
            x='time',y='P3a',hue='attention',style='expectation',
            errorbar='se',ax=axes[0]) 
    sns.lineplot(data=p3b_win[p3_df['subject']==i],
            x='time',y='P3b',hue='attention',style='expectation',
            errorbar='se',ax=axes[1]) 
    for i, ax in enumerate(axes.flat):
        ax.axhline(y=0, lw=1, c='black' )
        ax.axvline(x=.3,ls='--',lw=1, c='black' )
        ax.axvline(x=.4,ls='--',lw=1, c='black' )

    plt.tight_layout()
    plt.show()

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(8, 8))
sns.lineplot(data=p3_df,
        x='time',y='P3a',hue='attention',style='expectation',
        errorbar='se',ax=axes[0]) 
sns.lineplot(data=p3_df,
        x='time',y='P3b',hue='attention',style='expectation',
        errorbar='se',ax=axes[1]) 

axes[0].axhline(y=0, lw=1, c='black' )
axes[0].axhline(y=0, lw=1, c='black' )
axes[0].axvline(x=.3,ls='--',lw=1, c='black' )
axes[0].axvline(x=.4,ls='--',lw=1, c='black' )
axes[1].axvline(x=.35,ls='--',lw=1, c='black' )
axes[1].axvline(x=.45,ls='--',lw=1, c='black' )

plt.tight_layout()
plt.show()

In [5]:
""" 
P1
"""
p1_df = pd.DataFrame({'attention':df_1.attention,'expectation':df_1.expectation,'position':df_1.position,
                      'time':df_1.time, 'visual_field':df_1.visual_field,'subject':df_1.subject,
                      el_1:df_1.baseline_corrected, el_2:df_2.baseline_corrected,
                      el_3:df_3.baseline_corrected, el_4:df_4.baseline_corrected,
                      el_5:df_5.baseline_corrected, el_6:df_6.baseline_corrected})
# Create contra and ipsi lateral selection for p1
p1_df['contra'] = p1_df.apply(
    lambda row: (row['PO4'] + row['PO8'] + row['O2'])/3 if row['position'] in [3,4] else (row['PO3'] + row['PO7'] + row['O1'])/3,
    axis=1
)


In [ ]:
# Window selection
p1_tmin = 0.10
p1_tmax = 0.16
p1_timwid = 0.02
p1_sampwid = int(p1_timwid*512)
p1_meas = 'contra'

# Select the place to search for peak
p1_slect = p1_df[(p1_df['time']>p1_tmin) & (p1_df['time']<p1_tmax)]
collaps_p1 = p1_slect.groupby(['subject','time'])[p1_meas].max().reset_index()
p1_peak_lat = collaps_p1.loc[collaps_p1.groupby('subject')[p1_meas].idxmax(), ['subject', 'time']]

# Find time point in main df
idx_p1 =[]
for id,row in p1_peak_lat.iterrows():
    sub = row.subject
    time_point = row.time
    # Find matching rows in df
    matches = p1_df[(p1_df['subject'] == sub) & (p1_df['time'] == time_point)].index.tolist()
    idx_p1.extend(matches)
winidx = []
# Select window around it
for i, idx in enumerate(idx_p1):
    win_li = np.arange(idx - p1_sampwid, idx + p1_sampwid +1)
    winidx.extend(win_li)

# Apply to df
p1_win = p1_df.iloc[winidx]
p1_means= p1_win.groupby(['subject','attention','expectation'])[p1_meas].mean().reset_index()

In [ ]:
destination_folder = r"C:\Users\mvmigem\Documents\data\project_1\compiled_dataframes/"
p1_means.to_csv(destination_folder + "p1_oc_df.csv")

In [11]:
for i in subject_list:
    fig, axes = plt.subplots(figsize=(8,6))
    sns.lineplot(data=p1_df[p1_df['subject']==i],
                x='time',y='contra',hue='attention',style='expectation',
                errorbar='se') 

    plt.tight_layout()
    plt.show()


C:\Users\mvmigem\AppData\Local\Temp\ipykernel_19224\145068437.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(figsize=(8,6))


In [13]:
fig, axes = plt.subplots(1,1,figsize=(12, 6))
sns.lineplot(data=p1_df,
            x='time',y='contra',hue='attention',style='expectation',
            errorbar='se') 

axes.axhline(y=0, lw=1, c='black' )
axes.axvline(x=.1,ls='--',lw=1, c='black' )
axes.axvline(x=.16,ls='--',lw=1, c='black' )
plt.tight_layout()
plt.show()

In [18]:
"""
P2
"""
# Window selection
p2_tmin = 0.21
p2_tmax = 0.29
p2_timwid = 0.025
p2_sampwid = int(p2_timwid*512)
p2_meas = 'contra'

# Select the place to search for peak
p2_slect = p1_df[(p1_df['time']>p2_tmin) & (p1_df['time']<p2_tmax)]
collaps_p2 = p2_slect.groupby(['subject','time'])[p2_meas].max().reset_index()
p2_peak_lat = collaps_p2.loc[collaps_p2.groupby('subject')[p2_meas].idxmax(), ['subject', 'time']]

# Find time point in main df
idx_p2 =[]
for id,row in p2_peak_lat.iterrows():
    sub = row.subject
    time_point = row.time
    # Find matching rows in df
    matches = p1_df[(p1_df['subject'] == sub) & (p1_df['time'] == time_point)].index.tolist()
    idx_p2.extend(matches)
winidx = []
# Select window around it
for i, idx in enumerate(idx_p2):
    win_li = np.arange(idx - p2_sampwid, idx + p2_sampwid +1)
    winidx.extend(win_li)

# Apply to df
p2_win = p1_df.iloc[winidx]
p2_means= p2_win.groupby(['subject','attention','expectation'])[p2_meas].mean().reset_index()

In [19]:
fig, axes = plt.subplots(1,1,figsize=(10, 6),sharey=True,sharex=True)

sns.barplot(p2_means,y='contra',x='attention',hue='expectation',edgecolor= 'none', palette= [my_palette[4],my_palette[8]],
            errorbar='se', errwidth= 5,
            capsize=.1,width=.4, linewidth=0.7)

plt.tight_layout()
plt.show()

C:\Users\mvmigem\AppData\Local\Temp\ipykernel_19224\958935651.py:3: FutureWarning: 

The `errwidth` parameter is deprecated. And will be removed in v0.15.0. Pass `err_kws={'linewidth': 5}` instead.

  sns.barplot(p2_means,y='contra',x='attention',hue='expectation',edgecolor= 'none', palette= [my_palette[4],my_palette[8]],


In [20]:
destination_folder = r"C:\Users\mvmigem\Documents\data\project_1\compiled_dataframes/"
p2_means.to_csv(destination_folder + "p2_oc_df.csv")